In [5]:
import numpy as np
sim = msp.simulate(sample_size=2, length=L, mutation_rate=1e-8, recombination_rate=1e-9, Ne=1e4)
seq = np.zeros(L, dtype='u1')
positions = np.array([v.position for v in sim.variants()])
seq[positions.astype(int)] = 1
gaps = np.diff(positions)

In [7]:
import msprime as msp
L = 1_000_000
sim = msp.simulate(sample_size=2, length=L, mutation_rate=1e-8, recombination_rate=1e-9, Ne=1e4)

import numpy as np
x = []
y = []
i = 0
while i < 10007:
    L = 1_000_000
    sim = msp.simulate(sample_size=2, length=L, mutation_rate=1e-8, recombination_rate=1e-9, Ne=1e4)
    positions = np.array([v.position for v in sim.variants()])
    gaps = np.diff(positions)
    if len(gaps) <= 100:
        continue
    x.append(gaps[:50])
    y.append(gaps[50:100])
    i += 1
    
x=np.array(x)
y=np.array(y)

In [23]:
# Get one long sequence 
import numpy as np
i = 0
corpus = []
while i < 5000:
    L = 1_000_000
    sim = msp.simulate(sample_size=2, length=L, mutation_rate=1e-8, recombination_rate=1e-9, Ne=1e4)
    positions = np.array([v.position for v in sim.variants()])
    gaps = np.diff(positions)
    if len(gaps) > 1000:
        corpus.append(gaps)
    i += 1
corpus=np.array(corpus)
corpus.shape

(1, 1132)

In [27]:
np.savetxt("/Users/peng/Desktop/Gene_research/Genseq.txt", corpus,delimiter=',')

In [22]:
print("x shape:",x.shape)
print("y shape:",y.shape)

x shape: (5000, 50)
y shape: (5000, 50)


In [27]:
# flatten x to compute number of unique numbers
        
def top_num(x,y):
    flattened = []
    for sublist in x:
        for val in sublist:
            flattened.append(val)

    flattened_y = []
    for sublist in y:
        for val in sublist:
            flattened_y.append(val)
    
    top = 0
    top = max(max(flattened), max(flattened_y))
    return top

In [28]:
print("max value in x:",max(flattened))
print("max value in y:",max(flattened_y))
print(top_num(x,y))

max value in x: 370282.9671637656
max value in y: 410573.1911046213
410573.1911046213


In [4]:
x[0]

array([ 440.98916468,  708.85193645,  491.03451468,  162.67687766,
       3635.91610171, 1898.79648583,  745.71080997,  282.66494379,
       1351.07065688,  316.17803887,  426.11580737,  323.67358783,
        523.90315051,  266.71941932, 1098.65226744, 2809.43993762,
        476.86280551, 1141.21526702,  515.23499611,  400.54924577,
        156.87796986, 1473.974412  ,   83.80657142,  562.86770954,
       1030.53364366,  163.94432049,   76.60956439, 2370.97711162,
        868.36684356, 3043.22318916,  120.51624077, 1347.20778079,
       2618.26785486,  447.59704267, 3057.49840218, 3230.37790138,
        777.5921285 ,  616.24722602, 2881.7761304 , 1582.97307948,
       2263.5372434 ,  884.05845368,  950.67238051,  391.19517475,
       1360.20287021,  519.50072365, 2291.29195493,  785.92775552,
        932.24776841,  495.16402436])

In [5]:
y[0]

array([1.36113512e+03, 2.08596703e+02, 1.32026640e+03, 2.05246535e+03,
       1.64380977e+02, 1.52989251e+03, 2.53147235e+02, 1.61062328e+03,
       4.38016586e+00, 1.69247734e+03, 8.30554959e+03, 9.67658820e+02,
       1.57163517e+03, 4.35037946e+03, 3.99516569e+03, 1.43049889e+03,
       8.55061849e+02, 1.31958156e+02, 1.70789168e+03, 3.05887263e+02,
       2.61561900e+02, 6.29696603e+03, 1.49203400e+02, 1.46946394e+03,
       2.60264320e+03, 1.53776930e+03, 8.39698119e+02, 3.38859204e+03,
       3.35407749e+03, 7.60133974e+02, 1.76788313e+03, 1.68681423e+03,
       7.60025268e+02, 2.58467169e+02, 1.77397519e+01, 1.63487373e+03,
       9.16074803e+02, 9.27893027e+02, 1.52344070e+03, 1.44598476e+03,
       2.09758995e+03, 1.80733925e+00, 5.03610295e+02, 6.06383764e+02,
       2.44795766e+03, 3.34264912e+03, 1.41191138e+03, 7.16927310e+00,
       5.18213150e+03, 1.87276639e+03])

In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
from __future__ import print_function, division
from music21 import converter, instrument, note, chord, stream
from keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_discriminator(self):

        model = Sequential()
        model.add(CuDNNLSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(CuDNNLSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load and convert the data
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)

            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
              self.disc_loss.append(d_loss[0])
              self.gen_loss.append(g_loss)
        
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        # Get pitch names and store in a dictionary
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes = [int_to_note[int(x)] for x in pred_notes]
        
        create_midi(pred_notes, 'gan_final')
        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

if __name__ == '__main__':
  gan = GAN(rows=100)    
  gan.train(epochs=5000, batch_size=32, sample_interval=1)

In [28]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
#         fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=4000, batch_size=32, save_interval=50)

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)         

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.172230, acc.: 29.69%] [G loss: 0.713075]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


FailedPreconditionError:  Error while reading resource variable _AnonymousVar56 from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/_AnonymousVar56/N10tensorflow3VarE does not exist.
	 [[node mul_56/ReadVariableOp (defined at /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_keras_scratch_graph_5993]

Function call stack:
keras_scratch_graph


In [44]:
from keras.utils import np_utils
# x=[4.514035720763564314e+03,4.627780786182374868e+02,1.020945567298307651e+03,1.070354155058161268e+03,5.533543765011305368e+03,7.590369377976319811e+03,2.954948118591055390e+02,1.466116552641607996e+03,4.295811526119941846e+02,3.037189918146104901e+02,2.405418348353123292e+00,1.099691661965771345e+02,2.415395419985645276e+03,9.602713470572780352e+01,2.637622617742672446e+02,3.883147210605675355e+02,4.103439594055525959e+02,3.551888462248534779e+02,8.538832430593902245e+01,5.905625803497459856e+02,2.756134854054544121e+02,3.337264630229779868e+03,8.452409671609348152e+01,2.437427593082102248e+03,9.041067672946956009e+02,2.004874771996401250e+02,1.970176135431684088e+03,3.962430043657761416e+03,7.125869599815923721e+02,4.855300338300876319e+02,3.314717316398891853e+02,1.059469240046746563e+01,4.242414881729418994e+02,6.540902069531803136e+02,8.888835177637010929e+02,1.905628445429269050e+03,1.939418279724559397e+02,2.409704104248448857e+02,1.171866432832612190e+02,3.603392362044833135e+02,3.392229437393543776e+02,4.800876806979213143e+02,1.868073045227356488e+02,2.047662194704484136e+03,9.335884379670460476e+01,6.273468676169286482e+02,5.176326141159806866e+02,8.716325789734837599e+02,1.899757382977986708e+02,3.242648889347765362e+01,3.001338812762398447e+03,1.139496040585494484e+03,3.619060527434339747e+01,2.357869184323644731e+02,1.580632874724105932e+02,6.886942157018784201e+02,7.261416926722158678e+02,7.332367527834576322e+02,3.641520747695176397e+02,8.100511807978764409e+02,3.435326495320623508e+02,1.690902797880567960e+03,5.928782385333179263e+02,6.515067413727665553e+02,8.467909570205956697e+02,3.664815713596472051e+02,7.888356199778354494e+02,2.540862429916014662e+03,2.311798521774588153e+03,1.156491994587147201e+04,8.702659064975305228e+02,1.378849131578463130e+03,8.781301760279166047e+02,4.899542435430368641e+03,1.762194796834955923e+01,5.304347161584009882e+02,2.954088241670877323e+03,6.556641254791320534e+02,2.527520428248040844e+03,5.082523811404826120e+02,1.273265805216215085e+02,2.719441184785100631e+03,1.076596799920895137e+03,1.525972824835625943e+03,6.790465513993549393e+02,1.914899587620180682e+03,6.180996023298212094e+02,3.349306768108799588e+02,3.622473592098976951e+02,1.898613516426557908e+02,1.097605728838243522e+03,5.713236811690876493e+02,1.790149277676973725e+02,9.734968373278388754e+02,7.004977495111961616e+02,8.814141016272769775e+02,7.158871537680242909e+02,1.809466204780619591e+03,1.501168348329578293e+03,1.188461714318356826e+03,6.460041785302746575e+02,1.907131076148507418e+02,1.698771777267975267e+02,6.098369566619076068e+02,1.166939044858489069e+03,1.150211613165010931e+03,5.164379453997244127e+02,1.986817527438252000e+03,2.545918901334225666e+02,2.207884136669919826e+03,2.084374972916266415e+03,2.345823669692035764e+02,6.263489316216146108e+02,2.889527510898187757e+01,3.528329778349143453e+02,3.255984583902172744e+02,1.460467251021385891e+03,1.698072249128017575e-01,4.613832089953939430e+02,1.124346450374810956e+03,2.152490253082942218e+02,1.132284235596336657e+03,1.508698308151680976e+03,9.066797533264616504e+01,3.415347696267708670e+02,1.328122311313054524e+02,1.724447627658664715e+02,3.457992699387250468e+01,1.052676101019460475e+03,2.821636285328422673e+02,4.789414677410968579e+02,1.313472388279507868e+03,1.002996961087454110e+03,6.826412439288105816e+02,5.483831797902821563e+02,4.167605321343144169e+03,3.246161422948352993e+01,6.409270781479426660e+01,3.461576318026782246e+03,1.276960499419161351e+03,1.832854374675662257e+03,1.244247940773231676e+03,1.618160881987423636e+02,1.339542167342442553e+02,1.607073702674562810e+03,2.001484968064760324e+02,8.284450839911005460e+01,2.066421832420950523e+03,6.706515818813932128e+02,1.625680882008571643e+03,4.340712544186681043e+02,1.825807617754326202e+02,2.253831429743731860e+02,5.153518893284490332e+02,1.083830088635237189e+03,3.706201705317944288e+02,3.230852592153532896e+02,2.308305738840135746e+02,4.844912676578387618e+02,4.936684104982996359e+02,2.479634970144834369e+02,5.845089472446707077e+02,3.445180908468319103e+02,2.997544425553642213e+02,9.271339487755612936e+02,1.420842508945555892e+03,1.689639170160196954e+03,8.027711526587954722e+02,2.243283487407345092e+03,6.743882763420988340e+02,3.625110717391362414e+02,8.744701840225025080e+02,1.564875939774676226e+03,1.292449442786164582e+02,2.275970795388566330e+02,4.299019908123009373e+02,9.536760389205301180e+01,1.038448294767062180e+03,3.838891148541006260e+01,5.265925297065696213e+02,6.964564760752546135e+02,1.447708748379343888e+03,2.093611843797581969e+03,3.452833311536815017e+02,2.295524548738321755e+02,3.517710376985196490e+02,1.087027129399153637e+03,2.585590013562090462e+03,1.071859660806774627e+03,2.234557444473641226e+03,2.639447791806596797e+02,3.813074002774665132e+02,2.596610621594300028e+02,2.329551177133544115e+03,6.522078014508588240e+02,5.514512655635480769e+02,4.305429484365158714e+02,1.056566368161002174e+03,2.011572868056828156e+02,4.731197271125856787e+03,1.049734920298738871e+03,4.645665962755447254e+01,4.840487307253060862e+03,5.134175032427301630e+02,1.499624261003336869e+02,1.221189358968869783e+03,3.981706250391725916e+02,1.432167868819669820e+01,3.306036813883110881e+02,6.354951902462053113e+02,7.774318470693542622e+02,1.128596305550046964e+03,1.858022182770946529e+02,2.639640925376443192e+02,9.747561133364797570e+01,6.781908678071631584e+02,6.894310467991745099e+02,8.196446086671203375e+02,8.648903985919896513e+02,1.979587345937034115e+01,5.116306763486645650e+02,2.157100286354252603e+02,8.545340037029236555e+02,8.024415675493655726e+02,9.326213627420074772e+02,8.359557416639290750e+02,1.103259605555213057e+02,3.640638279747130582e+03,1.894465161109517794e+03,2.012994121490046382e+02,2.280180167076759972e+03,5.779021605422574794e+03,1.145041307274368592e+01,5.633432444715581369e+02,7.853117061418015510e+02,4.140531079811044037e+02,2.920177636296139099e+02,3.140269567494397052e+02,3.936067386095819529e+02,7.613171728881716263e+02,3.010649196279991884e+02,9.744394628680311143e+00,2.537170649358886294e+02,2.010553649145876989e+02,7.906695144966943190e+01,4.012836914883810095e+02,1.560698782434046734e+02,1.660791211409377865e+01,2.431413982641533948e+02,2.029338515923009254e+02,4.928263396921975072e+02,9.959913561528082937e+00,1.328747729411814362e+03,3.067678470183745958e+02,8.232552385373855941e+01,9.725630861532408744e+01,7.847180415185284801e+02,1.506920286467939150e+02,2.429699662883649580e+01,1.800181569811829831e+02,7.532825213627947960e+02,6.958000551356235519e+00,4.736777570446429309e+02,1.373237444265105296e+02,1.172011551639909158e+03,4.691875248705036938e+01,8.375585463448078372e+01,4.680986876276438124e+01,6.048515789912198670e+01,3.517705856486863922e+02,1.259530724877840839e+02,5.812656322001712397e+02,2.396123797768959776e+02,5.796993960912222974e+02,2.033441224546986632e+02,8.117440653823432513e+02,5.520299457741202787e+01,8.017464614785858430e+02,1.148397756643535104e+02,1.448192456427204888e+02,1.985817126771144103e+02,2.252550864406803157e+02,3.928333729008736555e+02,3.974837232328427490e+02,1.291631600745313335e+02,7.393500322971667629e+02,2.949208194112870842e+02,1.638526550561015029e+03,1.531186621233879123e+03,1.172115318119875155e+03,5.392293946337886155e+02,3.505681473026052117e+02,5.337368133759591728e+02,5.071718002289999276e+02,7.253002092700335197e+02,9.347005189087940380e+01,8.800398246045806445e+02,2.418228135848068632e+03,1.167282942915917374e+03,1.584569533932721242e+03,3.456241740082623437e+02,4.600443391027278267e+03,2.102278372648288496e+03,3.651104458191548474e+02,5.605815065447182860e+03,7.990194910013349727e+03,1.409949904383800458e+03,7.445638533213059418e+02,5.645190171285066754e+02,8.556593353245407343e+02,6.965583224734291434e+01,1.081506203301483765e+03,8.895273944073705934e+02,2.436118316561332904e+02,5.544482259940123186e+02,4.887831253553740680e+02,1.214338964314665645e+02,4.095114040703629144e+02,9.452153902581194416e+02,2.855069600146380253e+02,2.483983593486482278e+01,1.562434148993925191e+02,6.875831165833515115e+02,2.587248381890822202e+02,7.847806990759563632e+02,3.683644838363397866e+01,9.534360750142950565e+02,8.390103066980373114e+01,6.990721378277521580e+01,9.999739448503823951e+02,3.583268854090711102e+02,7.604801400345750153e+02,8.625940131151583046e+01,6.874050769052701071e+01,9.091544837999390438e+01,6.885531801416072994e+01,3.859944241282646544e+02,3.134115283312276006e+02,3.841718697138130665e+02,6.383395288954488933e+00,3.430786835396429524e+01,4.610968222155352123e+02,6.127850417184527032e+02,1.795325291467015631e+02,7.266612505602533929e+02,1.013051197680470068e+03,2.580720412032678723e+03,6.420088288438273594e+02,3.269687427291646600e+02,5.381510709480498917e+02,1.036214860595995560e+03,3.640336307911202312e+02,9.538444093830185011e+01,8.219061742202611640e+02,2.671831197148421779e+02,4.320767708161729388e+02,5.225289318296127021e+02,8.359596907207742333e+02,7.513527027864474803e+02,6.167111120847868733e+03,7.799018970204051584e+02,2.261381779065937735e+03,4.451071156072430313e+00,9.896173455375246704e+02,5.450433489513816312e+02,2.161837024876556825e+03,6.569173902203910984e+02,6.633270198926329613e+02,1.084291128222655971e+03,2.252149519236409105e+02,4.440111581960809417e+02,1.405924125719117001e+03,4.182735622033942491e+01,1.009681621590279974e+02,1.127791247456683777e+03,3.003133427696884610e+02,4.007552133270655759e+02,9.603262389513547532e+02,3.449046510593616404e+02,5.706726496643386781e+01,2.530473042579833418e+02,3.071522011794149876e+01,6.813119602706283331e+02,6.181789611568092369e+02,4.913717925205128267e+01,5.452515375112416223e+02,1.540264809486689046e+02,1.392627450444852002e+02,3.423049778555287048e+02,2.306517663956037723e+02,2.339069741083076224e+02,1.236583820095984265e+01,3.169503358512884006e+02,7.490049123030621558e+01,3.910108948052511550e+02,3.295484657705528662e+02,4.851950346740195528e+02,9.533350034470204264e+02,1.527332359007559717e+02,1.391772460866486654e+02,6.012288692597649060e+02,1.656365534957731143e+02,1.574592398823006079e+02,1.852629565680399537e+01,5.432897150923963636e+01,1.227419010094599798e+02,8.478484138846397400e+02,1.636944101167027839e+02,4.009389162399456836e+02,3.568843478664639406e+02,6.579887727400055155e+02,8.560283411704585887e+02,4.286433898766408674e+02,3.584451947077177465e+02,2.301937872627167962e+02,1.746340610600309446e+03,2.362615995981614105e+02,1.136412024759338237e+02,3.878395604737452231e+02,1.707309384359803516e+03,6.148017466163728386e+02,5.523439575251541100e+02,7.135630195291014388e+01,3.319342620146926492e+01,1.899502624816959724e+02,1.863477068306528963e+02,1.420994177701475564e+03,1.165770728530536871e+03,2.514126786345732398e+02,6.343524750744109042e+02,6.311990651758387685e+02,1.461167558629531413e+02,3.631992828566581011e+00,1.005589970183849800e+03,1.898989545982913114e+02,4.457315191430971026e+02,1.726597723963786848e+02,7.886878530912217684e+02,2.075596865499974228e+02,1.383332238399307244e+02,1.380812207026174292e+02,2.664736631929758005e+03,1.021710874727519695e+03,3.374511205620365217e+01,8.240576451295055449e+01,8.430209631777834147e+02,5.128724136753589846e+02,4.563762925887422170e+03,1.336377707909559831e+02,1.511048304688883945e+03,3.065936410823429469e+03,2.650720535037864465e+03,4.806324710137560032e+02,5.483906955506536178e+02,4.249724914645776153e+02,3.425002022126573138e+02,2.047377300731022842e+02,4.163851778804091737e+02,1.550435744114220142e+02,7.907025588471442461e+03,8.112852066207560711e+02,9.021790692663635127e+02,3.737969281621626578e+03,2.222383138860575855e+03,3.814855479437101167e+03,1.757384280863392632e+03,7.466207868263591081e+02,9.127151421230519190e+02,1.954157212073798291e+03,1.084471071491373004e+04,5.352122346702264622e+03,7.726608758135698736e+02,7.636673052014666609e+02,6.016625992365879938e+01,3.342790615825331770e+03,5.693820764936390333e+02,3.578401637758885045e+03,7.039262283232528716e+01,1.408584258295659674e+04,3.247406030019395985e+03,5.643931431117409375e+03,4.636362591321812943e+01,2.053410605612327345e+03,2.000401386734680273e+03,8.169432265749201179e+02,2.946323516024276614e+02,9.752191460113972425e+02,1.602206554892705753e+03,2.580692696308018640e+03,2.474129394141666126e+03,3.003151634518522769e+03,9.058521468379767612e+02,2.767384541453793645e+01,2.525418811813520733e+03,3.304193350124405697e+02,2.352892589275725186e+02,1.708549472723621875e+03,3.214953822927782312e+02,5.097583610173314810e+02,1.440838336684391834e+03,5.464057725082966499e+03,9.577147078240057454e+03,2.055985286236973479e+03,2.614625594804529101e+03,9.720713788778986782e+03,1.089514118193881586e+03,3.180083555578778032e+03,6.201026614823436830e+03,1.317976788832223974e+03,1.595595367119298317e+03,3.365426187380217016e+02,1.111021880178130232e+03,3.369068327356362715e+02,1.847254513814114034e+02,3.877444936422398314e+03,6.678795424220152199e+01,5.164190368722192943e+02,1.511555455790366977e+02,1.071246625886764377e+03,3.772490734081482515e+03,8.003169649918563664e+02,6.752085207925643772e+03,1.682788229259196669e+03,5.567121454157051630e+03,5.509167628231691197e+03,2.064657360284239985e+03,3.877079347229213454e+03,7.039377342090941966e+03,1.429524017087765969e+03,2.639515991674619727e+03,7.942963476432487369e+02,4.803567479936173186e+02,3.118423914614948444e+03,1.381668763416819274e+02,1.426494639274082147e+03,3.576695782643510029e+02,6.253179099434055388e+02,1.470126788145862520e+02,6.840660667808260769e+02,1.344288468536804430e+03,3.333220268366858363e+02,8.223110156343318522e+01,8.856782370752189308e+02,1.716034861674648710e+03,4.079967965286923572e+02,3.156530868620611727e+02,1.665540015056496486e+03,9.201640527624404058e+02,5.781281116316094995e+02,7.577248539539286867e+02,6.230386264882981777e+02,8.965259130013873801e+02,3.243383987458655611e+02,1.073524057998787612e+03,1.380101018253481016e+02,2.342808722091140226e+03,3.740946601088857278e+03,5.278658012090018019e+02,1.266811286036856472e+03,4.185888718520873226e+03,1.371820583469117992e+03,4.326565261051291600e+02,2.768041229902184568e+03,1.227084422054584138e+03,1.104124662005458958e+03,1.214053137396695092e+02,2.786799437989131548e+03,2.305913477807538584e+03,5.955651503590634093e+02,7.974656832867767662e+02,5.176538139964686707e+02,7.128353422244545072e+02,5.078783585423370823e+02,2.326134222074761055e+03,9.949778653462417424e+02,2.665684022931382060e+02,1.323620487753767520e+03,1.078256848909892142e+03,1.188035205610794947e+02,1.252135452629416250e+03,5.958914294436108321e+02,1.068730056819273159e+03,5.005014879700029269e+02,1.595949313548393548e+03,6.221769159904215485e+01,1.739341270708246157e+02,1.090614061593310907e+03,8.269677258097799495e+02,4.769695420777425170e+02,3.014808114786865190e+02,2.038193515538587235e+03,2.522931249671382830e+02,6.674680568458279595e+02,5.309193604049505666e+02,7.880229289742419496e+02,1.851519429363310337e+03,3.336230206254404038e+02,4.457115161843830720e+02,6.392722413537558168e+01,9.379357078898465261e+02,7.090655713943997398e+02,1.639427018250338733e+03,9.947675732176285237e+02,6.706779863480478525e+02,4.154180429955013096e+02,1.261405400944175199e+03,4.035079789080191404e+02,2.487276577290962450e+03,4.721905315542826429e+02,4.919080760086653754e+02,2.873670744857867248e+03,3.635397382522933185e+01,8.855609568214276806e+02,6.508324645218672231e+02,1.563891722811269574e+03,8.718873322204453871e+02,3.332360558281652629e+01,3.738881397718796507e+02,3.321646512930747122e+02,3.205986535261617973e+02,1.193855381279718131e+03,2.030575105973985046e+02,1.502924802161287516e+02,5.175333132711239159e+01,8.284654938152525574e+02,9.466009470012504607e+02,8.734095504852011800e+02,2.710061979102902114e+02,6.291251175121869892e+02,1.996260352552519180e+03,1.026144018757855520e+03,2.862280237363651395e+03,1.844440043073846027e+02,1.728755174492835067e+03,6.658063626637449488e+02,3.408519302431959659e+02,1.899842732596443966e+02,4.201567537323571742e+02,8.800221201562089846e+02,4.939664965744595975e+03,1.217021633588243276e+03,1.006250359194236808e+03,1.676627846552291885e+02,9.629653233571443707e+02,1.102444257457391359e+03,9.126451553590595722e+01,3.262890948562300764e+03,1.592684036238351837e+02,3.280017602327279747e+02,8.384564007037552074e+02,7.260753818647935987e+01,3.193349609151482582e+00,6.408090186854824424e+02,2.402632580604404211e+01,1.783840088868746534e+02,2.941500443643890321e+02,3.634404245589394122e+01,4.131452004495076835e+01,5.771470279658678919e+02,3.306524124455871060e+03,7.565808005437720567e+02,3.585250467025907710e+02,1.266723879162687808e+03,7.288688666104571894e+02,9.772565860766917467e-01,1.947215357248205692e+02,5.301818058434873819e+02,3.880105669328477234e+02,4.288317570998333395e+01,1.331095862216316164e+02,1.258103281109360978e+02,6.201673464953200892e+02,7.094028560870792717e+02,1.213666112302104011e+03,1.896239648419432342e+02,3.002282629949040711e+02,1.952072952548041940e+01,7.090636159828864038e+01,1.629090987838339061e+03,6.176732402120251209e+02,1.710460136185865849e+02,5.957990180719643831e+02,1.458065728482324630e+02,1.012681384698953480e+01,9.019300358684267849e+01,1.413899347900878638e+02,4.149342284840531647e+02,1.268301026012049988e+02,2.910270768502959982e+02,1.080670470876386389e+03,3.976065976044628769e+01,2.581672358646756038e+02,1.888068985771387815e+02,1.139276868510525674e+03,5.465361379680689424e+01,7.162420761405956000e+02,1.623719272983726114e+02,2.083785486500710249e+01,4.624779136882862076e+02,7.386229448858648539e+02,1.190879127623047680e+02,2.373389523979276419e+02,1.284568338985554874e+03,6.986040078937076032e+02,1.281340617341920733e+02,7.361808659264352173e+01,2.414733448846964166e+02,8.919193803686648607e+02,1.354817001199349761e+03,8.662187150772660971e+01,4.339434947153786197e+02,3.820492149865021929e+02,3.605387502515222877e+02,1.512370822964934632e+03,7.677477004327811301e+02,2.212247906515840441e+02,1.948286008015275002e+02,1.711494351888541132e+03,4.063397326346021146e+02,7.922586347220931202e+01,1.725797217268263921e+02,8.858477138518355787e+02,2.991435606012819335e+02,2.650076512361411005e+02,6.159449955101590604e+02,2.271282027394045144e+02,4.517577697833767161e+02,2.528175453209551051e+02,1.421261589376372285e+03,2.860167204536264762e+02,2.264047327800653875e+00,3.669938058123225346e+02,1.311219831230351701e+03,6.095454475474543869e+02,8.424113501717802137e+02,4.888442379313055426e+02,4.577583936128066853e+02,2.473853212097892538e+02,2.146157151999650523e+02,3.668747366163879633e+02,6.962737543864641339e+02,5.794867799966596067e+00,1.170513603350846097e+02,4.648980381679721177e+02,3.121300021989736706e+02,3.691558959509711713e+02,1.258611480573890731e+02,2.702032876368612051e+02,3.209028438186505809e+02,2.003314302074722946e+02,8.872801950178109109e+02,8.507352691786363721e+02,1.469062750437296927e+01,4.040504642548039556e+02,1.712366238103713840e+03,3.990584956886013970e+02,3.426114053671481088e+02,8.291263021748745814e+02,3.791272161587839946e+02,2.288641382406814955e+03,1.232158485995256342e+03,3.410305142381694168e+02,1.089095491754822433e+03,7.119059710053261369e+01,3.385694799195043743e+02,6.164975862915161997e+01,3.799946583837736398e+02,6.110993875244166702e+02,6.197383271086728200e+02,7.181621567574329674e+02,3.987568090459099039e+02,2.767958025743719190e+02,2.208141638575936668e+03,2.809053443688899279e+02,5.812675736809615046e+02,1.050523799286806025e+03,8.367444877687375993e+01,8.143353185312589630e+02,5.875086532779969275e+02,3.948239024800714105e+02,2.180520395515486598e+01,7.319400809870567173e+02,3.272153195198625326e+02,1.286731742604169995e+02,6.425518129968550056e+02,2.975551162414485589e+02,1.692837857763515785e+02,6.139193472976330668e+02,5.438932529877638444e+02,4.178857257418567315e+02,6.103836513857822865e+01,6.617257887759478763e+02,2.993265994754619896e+02,5.914521722903009504e+01,2.088112967819790356e+03,4.087630764965433627e+01,2.515240571871981956e+03,1.764181120435241610e+02,1.123123143319739029e+03,2.180956873320974410e+01,2.307541528791189194e+02,3.382524313968606293e+02,9.904065179205499589e+02,7.931578855791594833e+02,2.461585105265257880e+02,1.861975376926129684e+02,1.665616260198876262e+03,1.273900514381239191e+03,9.709242729910183698e+02,1.503191593172959983e+03,1.012339591948431917e+03,5.804949493235908449e+02,1.370246273421216756e+02,1.318321834888774902e+02,1.560056395370163955e+03,4.015337818304542452e+02,2.793621538778534159e+02,3.046918467503273860e+02,2.302902878897730261e+02,3.249027089229784906e+02,2.124904380530351773e+02,3.674338215511525050e+02,2.203227923637023196e+02,8.400074181205127388e+01,4.178053909983718768e+02,1.161778475216124207e+01,1.043845162544166669e+02,1.289678225802490488e+02,2.657457660487852991e+01,2.310355658989865333e+02,3.299200614334549755e+02,4.022284566133748740e+01,1.596162785351043567e+02,1.762020232037175447e+02,5.944799861032515764e+01,1.259687859227415174e+02,9.853562134181847796e+02,5.797768809100380167e+02,4.855298052529105917e+02,3.448456183803500608e+02,7.914124817994888872e+01,6.137703214504290372e+01,1.670000942740589380e+02,5.420697338738245890e+02,7.123551299329847097e+01,4.583906833863584325e+02,2.805215862096520141e+02,1.146627587453695014e+02,1.416335157435736619e+03,2.841836448289686814e+02,4.028478826908394694e+02,2.350719019524985924e+02,4.060421613056678325e+01,4.726002174965105951e+02,6.011535905848722905e+01,2.685319313646759838e+02,5.961708309652749449e+01,9.368044126126915216e+01,1.533187262369319797e+02,1.580600912356749177e+02,1.213456565777305514e+02,4.259226538846269250e+01,8.337241356493905187e+00,9.019259564545936882e+02,3.646452356420923024e+01,1.317373174407985061e+02,4.525165039545390755e+01,1.086486204341053963e+01,4.937930491361767054e+02,8.023265969532076269e+02,1.646439768320415169e+02,5.136897966521792114e+01,8.020529248800594360e+02,1.481299321085680276e+02,2.750717984740622342e+02,5.226869923644699156e+00,3.213570832862751558e+02,9.899724032732192427e+02,9.301049515767954290e+01,3.965183902839198709e+02,3.756488053477369249e+00,3.884481859328225255e+02,7.990109641267918050e+01,2.765895432771649212e+02,1.393744270505849272e+02,1.465899945701239631e+02,1.682840774136129767e+02,6.402543799264822155e+01,1.866015833913930692e+03,7.616233606657478958e+02,1.764160726100672036e+02,8.049509758676867932e+01,2.073125982512719929e+02,1.806452051419764757e+02,5.318390422163065523e+01,4.166758801310788840e+01,2.212946882987162098e+02,2.123608428447041661e+02,1.099517568387091160e+01,8.767020363849587739e+01,1.536568526586052030e+02,2.673785017535556108e+01,1.735221819940488786e+01,1.060445463889045641e+02,1.625976818043272942e+01,1.546953807452227920e+02,1.957485451158136129e+02,8.098433326580561697e+01,4.742774183251895010e+02,1.016856944824336097e+03,3.702464712178334594e+01,6.295732955716084689e+01,3.556376285420265049e+01,1.071309720133431256e+02,1.067743805777630769e+03,1.319879187849233858e+03,2.928029943180736154e+02,1.566723162947106175e+03,1.730708684684941545e+02,4.127067354742903262e+02,3.568307499319780618e+02,1.314015581246931106e+02,3.150216657950310037e+03,7.913675373113946989e+02,4.051246247155359015e+02,2.207964028441347182e+01,1.494900623191148043e+03,1.154596344135934487e+02,3.389039248175686225e+02,5.819274774022633210e+02,3.559931307578226551e+02,3.706049405087949708e+03,9.459206038387492299e+00,1.619390547231305391e+02,1.517876301526557654e+03,1.587511495175072923e+03,8.620254842140711844e+02,1.001863796177785844e+03,7.940719012304907665e+02,8.652210541197564453e+01,3.393486680850619450e+02,1.288711727887974121e+03,5.235396450196858495e+02,1.652721491161501035e+02,1.250896959548117593e+02,1.485024743617745116e+03,1.168254290466778912e+03,2.451372980752494186e+02,2.108576602587709203e+02,2.372104019726859406e+03,1.872766405042493716e+02,5.233629763011122122e+02,9.209884229088202119e+02,7.858090132205979899e+02,4.618261733663966879e+02,8.187512114490382373e+02,4.752130484995432198e+02,3.129442923342459835e+03,6.281320393516216427e+01,4.372915679638972506e+02,4.040277094736229628e+02,5.207521821180125698e+02,1.383178952435730025e+03,4.829899201665539294e+02,1.160552768755238503e+03,1.380778460040455684e+03,1.417958584606647491e+03,5.196950594371883199e+02,2.843801214378559962e+03,9.623811363428831100e+01,5.436904078306397423e+02,1.018279823722550645e+02,8.953881027910392731e+01,5.318700724869268015e+02,8.938435639264062047e+02,2.601157362700905651e+02,3.812450302549405023e+02,4.653044847133569419e+02,5.662603472326882184e+02,2.239861202395055443e+03,1.347959732894087210e+02,4.186386130700120702e+02,1.648999046306125820e+02,2.337226168124238029e+03,2.173169491208391264e+02,6.282519292373908684e+02,4.741522437245585024e+02,4.358242099705385044e+02,2.976935920852702111e+01,1.185265226662158966e+01,3.069007186731323600e+01,3.474262966657988727e+01,1.017381584175280295e+03,3.749516128401737660e+02,3.280664962503360584e+02,5.714322486934252083e+02,1.408393102986272424e+03,1.028153707010205835e+02,7.067952815640019253e+02,6.781510392258642241e+02,9.247201137035153806e+01,3.801582057415507734e+02,3.120645441638771445e+02,1.041616836668225005e+03,8.973610398506280035e+02,1.920180301845539361e+02,2.356353083164431155e+02,3.419723560113925487e+02,3.281563626069109887e+02,3.234377265397924930e+02,1.682627564077265561e+02,2.031985830853227526e+02,2.327612483687698841e+02,6.855018801904516295e+02,2.023634182926034555e+02,3.156316762082278728e+02,2.654466390797169879e+02,4.000738650610437617e+02,7.288660895476350561e+02,6.522549226018600166e+01,6.284411861142143607e+01,1.422270912128733471e+02,1.304546245769597590e+01,1.200134860058315098e+02,6.885019591770833358e+02,8.827684683079132810e+02,7.988072171111125499e+02,2.143928416538983583e+00,3.623751241229474545e+02,3.936697511314996518e+03,1.250416369558311999e+03,6.229481967417523265e+02,8.167416631007799879e+02,3.158993896622443572e+03,2.681035822004312649e+03,4.502622836653608829e+01,3.648576472956920043e+02,4.434322584036272019e+01,1.142969728514668532e+03,6.022815788420848548e+02,1.850974977559759282e+03,3.199131724471226335e+02,2.293696089395089075e+03,5.527744474824285135e+02,5.661572558307088912e+02,5.967287963256239891e+01,3.677147250541020185e+02,7.341862188472878188e+02,2.018625054522417486e+02,1.340249904675874859e+02,8.188881266918033361e+02,2.684376175458310172e+02,6.613509828329551965e+01,5.764663849162170663e+02,5.334794707226101309e+02,2.614213000914314762e+02,2.896376412167446688e+02,8.979756207787431777e+00,1.103924460844136775e+02,1.453803781812079251e+03,6.630019148138817400e+02,1.794895483712898567e+02,1.061346408139215782e+02,2.707722363110398874e+03,1.004338032015948556e+03,1.055284710474894382e+03,8.145432749383617193e+02,9.476031024358235300e+01,1.424766561895608902e+03,1.245436804932774976e+03,1.160782990045845509e-01,1.235986692383885384e+02,5.379390223473310471e+02,6.778570872730342671e+02,5.472622508398490027e+02,6.339218740027863532e+02,3.400926072958391160e+02,5.361879429141990840e+02,1.351091415001545101e+03,5.543622272559441626e+02,2.507410847570979968e+03,1.999212610301095992e+03,4.045387350196251646e+02,4.117234376472188160e+02,6.066295394068583846e+01,5.292532532161567360e+01,9.258453455753624439e+02,1.130608880178770050e+02,1.166283442875836045e+03,5.004667685074964538e+02,1.498297533616656438e+03,9.868267967872088775e+02,1.073498464558389969e+03,2.121164016814786009e+03,9.852452967759454623e+02,3.383426055712625384e+02,4.251952645347919315e+02,1.530115335633745417e+02,3.475366139421472326e+03,7.270382242412306368e+02,2.201487038129242137e+03,2.742183506674482487e+03,5.691765096116578206e+02,1.078410814861999825e+03,1.644850617309566587e+03,5.668525741640478373e+02,2.726105661431443878e+03,9.902182811907259747e+02,3.150503233015304431e+03,6.870669298515422270e+02,4.242614038654603064e+03,2.558508081055479124e+02,4.421481423558318056e+03,6.191733880236279219e+02,7.694372498979908414e+03,2.054855137862032279e+02,4.674463524763705209e+02,4.649560111855389550e+02,9.752509211235446855e+02,1.467751919926959090e+03,1.076064661723794416e+02,8.716901212019147351e+02,1.256520727291353978e+03,1.806200162361958064e+03,5.698299217238556594e+03,1.872911685516941361e+03,1.301192655249731615e+03,2.670532133898581378e+03,5.733901106007397175e+01,5.160812607174739242e+01,1.491662885941797867e+03,1.894599938718369231e+03
# ]
n_vocab = len(set(x))
def prepare_sequences(seq, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100
    
    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(seq) - sequence_length, 1):
        sequence_in = seq[i:i + sequence_length]
        sequence_out = seq[i + sequence_length]
        network_input.append(sequence_in)
        network_output.append(sequence_out)

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1

    return (network_input, network_output)

n_input, n_output = prepare_sequences(x,n_vocab)

In [45]:
print(x)

[4514.035720763564, 462.7780786182375, 1020.9455672983077, 1070.3541550581613, 5533.543765011305, 7590.36937797632, 295.49481185910554, 1466.116552641608, 429.5811526119942, 303.7189918146105, 2.4054183483531233, 109.96916619657713, 2415.3954199856453, 96.0271347057278, 263.76226177426724, 388.31472106056754, 410.3439594055526, 355.1888462248535, 85.38832430593902, 590.562580349746, 275.6134854054544, 3337.26463022978, 84.52409671609348, 2437.4275930821022, 904.1067672946956, 200.48747719964013, 1970.176135431684, 3962.4300436577614, 712.5869599815924, 485.53003383008763, 331.4717316398892, 10.594692400467466, 424.2414881729419, 654.0902069531803, 888.8835177637011, 1905.628445429269, 193.94182797245594, 240.97041042484489, 117.18664328326122, 360.3392362044833, 339.2229437393544, 480.0876806979213, 186.80730452273565, 2047.6621947044841, 93.3588437967046, 627.3468676169286, 517.6326141159807, 871.6325789734838, 189.97573829779867, 32.426488893477654, 3001.3388127623984, 1139.496040585

In [46]:
n_input

array([[[4514.03572076],
        [ 462.77807862],
        [1020.9455673 ],
        ...,
        [1809.46620478],
        [1501.16834833],
        [1188.46171432]],

       [[ 462.77807862],
        [1020.9455673 ],
        [1070.35415506],
        ...,
        [1501.16834833],
        [1188.46171432],
        [ 646.00417853]],

       [[1020.9455673 ],
        [1070.35415506],
        [5533.54376501],
        ...,
        [1188.46171432],
        [ 646.00417853],
        [ 190.71310761]],

       ...,

       [[3936.69751131],
        [1250.41636956],
        [ 622.94819674],
        ...,
        [1301.19265525],
        [2670.5321339 ],
        [  57.33901106]],

       [[1250.41636956],
        [ 622.94819674],
        [ 816.7416631 ],
        ...,
        [2670.5321339 ],
        [  57.33901106],
        [  51.60812607]],

       [[ 622.94819674],
        [ 816.7416631 ],
        [3158.99389662],
        ...,
        [  57.33901106],
        [  51.60812607],
        [1491.66288594]]